In [1]:
import initialize

In [ ]:

# 1) create the optimization algorithm
# 2) decide on th


# x*1 + y* 2
# lambda x, y: x * 1 + y * 2
# Constaint = LTE(x=100, y=50) & GTE(x=0, y=0)
# genetic = Genetic()
# fitter.fit(func, constraint, x=20, y=30, const=['x'], penalty=-inf)
# fitter.step( state=state)
# 1) create the individual -> only x
# 2) populate
# 3) If not first
#    get children
#    cross over
#    elitism
# 3) mutate
# 4) assess
# 5) yield assessments

In [2]:
from zenkai import tansaku

x = 20
y = 30
populator = tansaku.RepeatPopulator(10)


objective = tansaku.FuncObjective(
    lambda x, y: 2 * x + 2.5 * y 
)

# how to convert to float
individual = tansaku.Individual(x=20.0, y=30.0)
population = populator(individual)

mutator = tansaku.GaussianMutator(0.2, 0.0)
population = mutator(population)

population = tansaku.Population(
    x=population['x'].clamp(0, 30),
    y=population['y'].clamp(0, 20)
)



print(population['x'])

# # Add a "Lambda" constraint
# LC(['x', 't'], lambda x, t: 2 * x + 3 * t < 4)
# LC() + 

objective = tansaku.FuncObjective(lambda x, y: 3 * x + 5 * y, penalty=1000)
assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

population = assessor(population)

divider = tansaku.FitnessProportionateDivider(6)
parents1, parents2 = divider(population)

print(parents1['x'], parents2['x'])
crossover = tansaku.SmoothCrossOverBreeder()
children =crossover(parents1, parents2)
print(children['x'])

elitism = tansaku.KBestElitism(2)
children2 = elitism(population, children)
print(children2['x'])


tensor([19.9816, 20.1212, 19.8699, 20.0561, 19.9204, 19.5393, 19.9489, 20.0806,
        19.8988, 20.0044])
tensor([19.9489, 19.8988, 19.8699, 19.5393, 19.5393, 19.8988]) tensor([19.9816, 20.0561, 19.8988, 20.0561, 20.0561, 20.0044])
tensor([19.9783, 20.0275, 19.8776, 19.6520, 19.8770, 19.9448])
tensor([19.5393, 19.8699, 19.9783, 20.0275, 19.8776, 19.6520, 19.8770, 19.9448])


In [ ]:
import typing
from zenkai import tansaku
from zenkai.kaku import Assessment, State
import torch
from functools import partial

class GeneticAlgorithm(tansaku.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.FitnessProportionateDivider(8)
        self.elitism = tansaku.KBestElitism(2)
        self.crossover = tansaku.SmoothCrossOverBreeder()
        self.mutator = tansaku.GaussianMutator(0.5, 0.0)
        self.populator = tansaku.RepeatPopulator(10)
        self.iterations = 100

    def optim_iter(self, objective: tansaku.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine((self, objective))
        assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = tansaku.Individual(**kwargs)
            population = self.populator(individual, state)
            population = self.mutator(population, state)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2, state)
                children = self.mutator(children, state)
                population = self.elitism(population, children, state)
            
            population = population.apply(partial(torch.clamp, min=0))

            population = assessor(population, state)
            my_state['population'] = population
            yield population.stack_assessments()
            i += 1


objective = tansaku.FuncObjective(
    lambda x, y: torch.abs(2 * x + 3 * y), constraint=tansaku.LTE(y=5, x=10) + tansaku.GTE(y=0, x=0), penalty=0, maximize=True
)


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, x=3.0, y=4.0):
    print(assessment)

    

In [2]:
import typing
from zenkai import tansaku
from zenkai.kaku import Assessment, State
import torch
from functools import partial

class GeneticAlgorithm(tansaku.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.FitnessProportionateDivider(8, 1)
        self.elitism = tansaku.KBestElitism(2)
        self.crossover = tansaku.SmoothCrossOverBreeder()
        self.mutator = tansaku.GaussianMutator(0.5, 0.0)
        self.populator = tansaku.RepeatPopulator(10)
        self.iterations = 100

    def optim_iter(self, objective: tansaku.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine((self, objective))
        assessor = tansaku.ObjectivePopAssessor(objective, ['w', 'b'], reduce_from=2)

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = tansaku.Individual(**kwargs)
            population = self.populator(individual, state)
            population = self.mutator(population, state)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2, state)
                children = self.mutator(children, state)
                population = self.elitism(population, children, state)
            
            # population = population.apply(partial(torch.clamp, min=0))

            population = assessor(population, state)
            my_state['population'] = population
            assessments = population.stack_assessments()
            yield population.stack_assessments()
            i += 1


import torch.nn as nn
from zenkai import ThLoss, IO

base_linear = nn.Linear(32, 8)
base_network = nn.Sequential(
    base_linear,
    nn.ReLU()
)

optimize_linear = nn.Linear(32, 8)
optimize_network = nn.Sequential(
    optimize_linear,
    nn.ReLU()
)

x = torch.rand(128, 32)
t = base_network(x)

objective = tansaku.NNLinearObjective(
    optimize_linear, optimize_network, ThLoss('MSELoss'), IO(x), IO(t)
)

# objective = tansaku.FuncObjective(
#     lambda x, y: torch.abs(2 * x + 3 * y), constraint=tansaku.LTE(y=5, x=10) + tansaku.GTE(y=0, x=0), penalty=0, maximize=True
# )


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, w=optimize_linear.weight, b=optimize_linear.bias):
    print(assessment)

    


        Assessment(maximize: False, value: tensor([[4.1713e-01, 3.4065e-02, 4.7586e-01, 4.0265e-01, 6.1298e-01, 4.4106e-03,
         7.1165e+00, 1.2615e+00],
        [6.5962e-03, 9.5766e-04, 2.9271e+00, 5.4407e+00, 1.5423e-01, 1.5760e+00,
         9.4295e-04, 1.2206e-01],
        [9.3303e-01, 4.8141e-03, 2.3949e-01, 4.3198e-01, 5.7765e+00, 5.8129e-02,
         9.4295e-04, 3.2075e+00],
        [4.0688e+00, 2.6135e+00, 2.3037e-02, 1.7014e-01, 4.8140e-02, 2.4781e-01,
         1.6635e-01, 0.0000e+00],
        [7.1967e-01, 2.1983e-01, 3.1986e+00, 3.1239e+00, 7.5610e-02, 1.5063e+00,
         9.4295e-04, 8.5597e-02],
        [3.1063e+00, 5.3407e-03, 6.1747e-02, 8.3564e-02, 4.1107e-01, 3.0909e-01,
         1.2224e-03, 1.9078e-04],
        [8.4363e-02, 9.5766e-04, 5.9203e-02, 6.6140e-02, 6.7925e-04, 2.2756e+01,
         1.6106e-02, 4.8308e-03],
        [1.1512e+00, 9.5766e-04, 6.4366e-02, 2.7949e-01, 6.9909e-03, 1.7269e-01,
         7.1174e-01, 6.9413e-02],
        [6.5962e-03, 6.8234e-01, 1.2

ValueError: Gather By dim must be less than or equal to the value dimension